Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
#from pandas import DataFrame, Series
#import csv    
#import gzip
#from IPython.core.pylabtools import figsize
#from IPython.display import clear_output
#figsize(8, 8)
#import re
#import time
import os
import glob
#%matplotlib inline               
#import matplotlib.pyplot as plt
#import seaborn as sns
#from matplotlib import cm
#from matplotlib.ticker import LinearLocator, FormatStrFormatter
#from mpl_toolkits.mplot3d import Axes3D
#plt.style.use('ggplot')

To connect Google drive to access data from it.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
''' UNZIPPING MAIN FILE AND ITS CONTENTS '''

import tarfile

path = '' # Enter your path here

In [ ]:
filename = os.path.join(path + '/Data/SMNI_CMI_TRAIN.tar.gz')
print(filename)
tar = tarfile.open(filename)
tar.extractall(path=path + '/Data/SMNI_CMI_TRAIN_unzipped')
tar.close()

/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN.tar.gz


In [ ]:
'''NAMING THE LOCATIONS'''

main_dir = cwd + "/Data/SMNI_CMI_TRAIN_unzipped/"
eeg_data = os.path.join(main_dir+'SMNI_CMI_TRAIN/')  # unzipped subjects data file location

subject_files = os.listdir(eeg_data) # list all the subject folders

In [ ]:
print(subject_files)

['co2a0000369', 'co2a0000365', 'co2c0000340', 'co2c0000339', 'co2a0000371', 'README', 'co2c0000344', 'csv', 'co2a0000370', 'co2c0000347', 'co2a0000378', 'co2c0000345', 'co2a0000377', 'co2a0000368', 'co2c0000338', 'co2a0000364', 'co2c0000342', 'co2a0000375', 'co2c0000341', 'co2c0000337', 'co2c0000346', 'co2a0000372']


## Parse Data

The following function will extract relevant parameters from each of the trial files belonging to the subject. The extracted files are stored in .csv format.

In [ ]:
def extract_info(filename):
    """ Function to parse individual trial details for a subject.
        Args:
        filename - The name of file to be parsed
        
        Return:
        subject_id - The subject ID of the subject
        alcoholic - Return True for alcoholic subject
        stimulus  - Returns the stimulus for a particular trial
        trial_num - Trial number of the subject. On average there is 90 trial files per 
                    subject"""

    with gzip.open(filename, 'rt') as f:
        reader = csv.reader(f)
        for idx, row in enumerate(reader):
            if idx == 0:
                m = re.match('# (co\d(a|c)\d\d\d\d\d\d\d)', row[0])  #match pattern for the subject name
                                                                     #contains 'a' for alcoholics 
                                                                     #and 'c' for control
                subject_id = m.group(1)
                alcoholic = (m.group(2) == 'a')
            if idx == 3:
                m = re.match('# (.*?) trial (\d+)', row[0] + row[1]) #matching trial number 
                stimulus = re.sub('\W', ' ', m.group(1))
                trial_num = int(m.group(2))
#             if idx == 4:
#                 m = re.match('# (.*?) chan (\d+)', row[0])
#                 chan_name = re.sub('\W', '', m.group(1))
#                 chan_no = int(m.group(2))
    return subject_id, alcoholic, stimulus,trial_num

In [ ]:
''' Converts files into CSV'''

def create_csv(subject_folder, main_dir, verbosity):
    """
        This function reads in individual trial files of subject folder, stacks the data
        in the form of a dataframe object and writes them out as CSV files.
        
        Args:
        subject_folder: The folder containing all the unzipped trial files of a subject
        main_dir: The main directory path to the data location
        verbosity: Pass a number greater than 1 for more output details
        
        Return:
        None 
    """
    data_dir = main_dir + subject_folder
    print('##########################################################')
    print('Beginning extraction of subject ', str(subject_folder))
    trial_dir = os.listdir(data_dir)
    trial_files = [file_i for file_i in trial_dir if file_i.endswith('.gz')]
    print('There are ', len(trial_files), 'trials under this subject')
    for file_no, file in enumerate(trial_files):
        tic = time.time()
        filename = data_dir + file
        df = pd.DataFrame()
        if verbosity == 0:
            print('Beginning extraction from file ', filename)

        sub_id, whether_alc, what_stimulus, trial_no = extract_info(filename)
        if verbosity > 1:
            print('Beginning extraction of file: ', filename)
            print('Subject id:', sub_id, ' Trial No: ', trial_no)
        with gzip.open(filename, 'rt') as f:
            reader = csv.reader(f, delimiter=' ')
            for idx, row in enumerate(reader):
                if row[0] != '#':
                    df = df.append(pd.DataFrame([row],columns=list(['trial_num', 'chan_name', 'epoch','voltage'])))
                        
                
            df = pd.concat([df,pd.DataFrame(columns=['subject_id', 'stimulus', 'alcoholic'])],
                           ignore_index=True)
            df[['subject_id', 'stimulus','alcoholic']] = [sub_id, what_stimulus, whether_alc] 
                
            out_name = main_dir + 'SMNI_CMI_TRAIN/' + sub_id + '_' + str(
                    trial_no) + '.csv'
            df.to_csv(out_name, index=False, sep='\t', encoding='utf-8')
            toc = time.time()
            
            if verbosity == 0:
                print('CSV file saved as ', out_name)
            elif verbosity > 1:
                print('There are ', df.shape[0], ' rows and ', df.shape[1],
                        ' columns in this dataframe object')
                print('CSV file successully saved as ', out_name)
                print('It will take %f mins more for this subject!' %
                        round(((len(trial_files) - float(file_no+1)) * float((toc - tic) / 60.0)),2))
                print('-------------------------------------------------------------------------------')
    print('All files extracted and saved.')
    print('###########################################################')
    return None

Start extraction of files for each subject. The extracted files will saved as csv files in the disk. 
The code proceeds only if files are missing. If all the files are satisfied the code prints the information with the number of trial files per subject. 
### NOTE: Subject co2c0000367 has error flags in the trial files and hence is avoided from the study.
 
### IMP: DO NOT RUN THIS CELL IF YOU DON'T WANT TO PARSE THE DATA(OR IF YOU DON'T HAVE THE FOLDER WITH ALL THE PARSED FILE IN THE DISK). 
THIS WILL TAKE A LONG TIME TO EXECUTE.

In [ ]:
len_trialfiles = []
for folder in subject_files:
    if(folder[0]!='c'):
      continue
    #print(folder)
    print(eeg_data)
    #path1 = eeg_data+folder
    #print(path1)
    os.chdir(eeg_data+folder)
    #print(os.chdir)
    #zipped_files = [zipfile for zipfile in glob.glob(path1+'/'+folder+'*.gz', recursive = True)]
    zipped_files = [zipfile for zipfile in glob.glob(folder+'*.gz', recursive = True)]
    #print(path1+'/'+folder+'*.gz')
    #path2 = main_dir+'SMNI_CMI_TRAIN/'
    #print(path2)
    os.chdir(main_dir+'SMNI_CMI_TRAIN/')
    #files = [file for file in glob.glob(path2+'/'+folder+'*.csv')]
    files = [file for file in glob.glob(folder+'*.csv')]
    len_trialfiles.append(len(zipped_files))
    #print(zipped_files)
    # check if all files are already extracted or not.
    if len(files) < len(zipped_files):
        create_csv("SMNI_CMI_TRAIN/"+folder+"/", main_dir, verbosity=5)
    else:
        print('All files satisfied for subject',folder,'; There are',len(files),'files.')

/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/
##########################################################
Beginning extraction of subject  SMNI_CMI_TRAIN/co2a0000369/
There are  30 trials under this subject
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000369/co2a0000369.rd.012.gz
Subject id: co2a0000369  Trial No:  12
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000369_12.csv
It will take 18.130000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000369/co2a0000369.rd.011.gz
Subject id: co2a0000369  Trial No:  11
There are  16384  rows and  7  columns

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000369_33.csv
It will take 9.360000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000369/co2a0000369.rd.015.gz
Subject id: co2a0000369  Trial No:  15
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000369_15.csv
It will take 8.410000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000369/co2a0000369.rd.03

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000365_5.csv
It will take 15.900000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000365/co2a0000365.rd.041.gz
Subject id: co2a0000365  Trial No:  41
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000365_41.csv
It will take 17.600000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000365/co2a0000365.rd.0

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000365_31.csv
It will take 7.520000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000365/co2a0000365.rd.012.gz
Subject id: co2a0000365  Trial No:  12
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000365_12.csv
It will take 6.920000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000365/co2a0000365.rd.04

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000340_7.csv
It will take 14.290000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000340/co2c0000340.rd.016.gz
Subject id: co2c0000340  Trial No:  16
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000340_16.csv
It will take 15.650000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000340/co2c0000340.rd.0

...

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000337_89.csv
It will take 7.770000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000337/co2c0000337.rd.047.gz
Subject id: co2c0000337  Trial No:  47
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000337_47.csv
It will take 7.130000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000337/co2c0000337.rd.05

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000346_14.csv
It will take 17.570000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000346/co2c0000346.rd.021.gz
Subject id: co2c0000346  Trial No:  21
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000346_21.csv
It will take 15.540000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000346/co2c0000346.rd.

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000346_29.csv
It will take 6.440000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000346/co2c0000346.rd.004.gz
Subject id: co2c0000346  Trial No:  4
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000346_4.csv
It will take 4.950000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2c0000346/co2c0000346.rd.018.

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000372_2.csv
It will take 13.300000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.029.gz
Subject id: co2a0000372  Trial No:  29
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000372_29.csv
It will take 15.090000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.0

There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000372_27.csv
It will take 5.140000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.041.gz
Subject id: co2a0000372  Trial No:  41
There are  16384  rows and  7  columns in this dataframe object
CSV file successully saved as  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000372_41.csv
It will take 4.560000 mins more for this subject!
-------------------------------------------------------------------------------
Beginning extraction of file:  /home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.01

## Functions for Concatenating Data for Single Subject

In [ ]:
def concat_df(sorted_files,verbosity):
    
    """
        Function to join together trial files belonging to a single subject
        and returns a dataframe object containing all the trials."""
    
    result = pd.DataFrame()
    for sorted_file in sorted_files:
        df = pd.read_csv(sorted_file, delim_whitespace=False, sep='\t', 
                         index_col = False)
        chan_names = df.chan_name.unique()    
        if verbosity >1:
            print('File loaded {}'.format((sorted_file.split('/')[5])))
        trial_voltage = []
        norm_voltage = []
        Norm_voltage =[]
        for chan in chan_names:
            #norm_voltage = []
            voltages = []  
            chan_df = pd.DataFrame()
            chan_df = df.loc[df['chan_name'] == chan]
            voltages = chan_df[['voltage']].values
            max_volt = max(voltages)
            min_volt = min(voltages)
            voltages = [x/abs(max_volt) if (x>0) else x for x in voltages]            #rescale voltage values to range [-1,1]
            voltages = [x/abs(min_volt) if (x<0) else x for x in voltages]
            norm_voltage.extend(voltages)
            Norm_voltage = np.asarray(norm_voltage)
        trial_voltage = np.concatenate(Norm_voltage, axis=0)
        df['norm_voltage'] = pd.Series(trial_voltage, index=df.index)
        result = result.append(df, ignore_index=True)
            
    return result

In [ ]:
os.chdir(main_dir+'SMNI_CMI_TRAIN/')
cwd = os.getcwd()
print(cwd)

/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN


In [ ]:
def sort_files(dir_path,sub_id):
    """
        Function sorts trial files under a subject folder in ascending order of trial number.
    """
    os.chdir(main_dir+'SMNI_CMI_TRAIN/csv/')
    files = [file for file in glob.glob(dir_path+sub_id+'_[0-9]*.csv') if file]
    #cwd = os.getcwd()
    #print(cwd)
    print(files)
    # sorting assumes that the file directory structure for a single trial file is of the format
    # '/home/user/eeg_data/eeg_csv/co2a0000372_107.csv'. If not change the position
    # and character used in the following line.
    sorted_files = sorted(files, key=lambda x: int((x.split('.')[0]).split('_')[7]))
    
    return sorted_files

In [ ]:
from pathlib import Path
import os.path
def create_SubCSV(sub_files):
    """
        This function calls the concat_df function and joins together indivdual trial files
        of the subject if the file does not exist. The dataframe object is written to disk as
        a pickle object.
    """
    
    
    out_dir = eeg_data + 'csv/'

    for subject in sub_files:
        #print(subject)
        #print(sub_files)
        if(subject[0]!='c'):
          continue
        outname = subject+'_full.pkl' 
        # file = Path(out_dir+outname)
        # try:
        #     file_path = file.resolve()
        #     print('File exists '+outname+'. Continue.')
                       
        # except FileNotFoundError:
        print('File does not exists '+outname+'. Begin.')
        sorted_files = sort_files(out_dir,subject)
        sub_df = concat_df(sorted_files,5)
        sub_df.to_pickle(outname)

## Create Subject files and store as pickle files
Create full subject files and store them to disk as pickle files for easy and fast access later.

In [ ]:
#os.listdir(eeg_data)

In [ ]:
#main_dir = "/content/drive/My Drive/SMNI_CMI_TRAIN_unzipped/"
eeg_data = os.path.join(main_dir+'SMNI_CMI_TRAIN/')  # unzipped subjects data file location

subject_files = os.listdir(eeg_data) # list all the subject folders
os.chdir(eeg_data)
alcoholics = [file.replace('/', '') for file in glob.glob('co?a*/')]
controls = [file.replace('/', '') for file in glob.glob('co?c*/')]
#print(alcoholics)
#print(subject_files)
create_SubCSV(controls)
create_SubCSV(alcoholics)

File does not exists co2c0000340_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000340_0.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000340_75.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000340_3.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000340_43.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000340_77.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000340_57.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000340_2.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000340_97.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File does not exists co2c0000344_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000344_11.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000344_43.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000344_26.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000344_28.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000344_71.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000344_45.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000344_24.csv', '/home/n

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File does not exists co2c0000345_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000345_16.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000345_13.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000345_7.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000345_29.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000345_14.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TR

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File does not exists co2c0000342_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000342_9.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000342_33.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000342_16.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000342_21.csv', '/home/nrvptl/Documents

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File does not exists co2c0000337_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000337_71.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c0000337_26.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2c00

...

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File does not exists co2a0000371_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2a0000371_14.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAI

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File does not exists co2a0000378_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2a0000378_12.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAI

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File does not exists co2a0000368_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2a0000368_0.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File does not exists co2a0000375_full.pkl. Begin.
['/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN/csv/co2a0000375_9.csv', '/home/nrvptl/Documents/python/EEG_Projects/Data/SMNI_CMI_TRAIN_unzipped/SMNI_CMI_TRAIN

File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects
File loaded EEG_Projects


In [ ]:
print("Number of alcoholics = {} and controls = {}".format(len(alcoholics),len(controls)))

Number of alcoholics = 10 and controls = 10
